In [1]:
import numpy as np
import pandas as pd
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr
    
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [3]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])
configs = [
    'zero_shot_no_heur',    
    'zero_shot_with_heur_hint_angle',
    'zero_shot_with_heur_hint_distance',
    'zero_shot_with_heur_hint_area',
    'zero_shot_with_heur_hint_angle_distance',
    'zero_shot_with_heur_hint_angle_area',
    'zero_shot_with_heur_hint_distance_area',
    'zero_shot_with_heur_hint_all',    
    'zero_shot_with_heur_value_angle',
    'zero_shot_with_heur_value_distance',
    'zero_shot_with_heur_value_area',
    'zero_shot_with_heur_value_angle_distance',
    'zero_shot_with_heur_value_angle_area',
    'zero_shot_with_heur_value_distance_area',
    'zero_shot_with_heur_value_all',    
    'few_shot_no_heur',    
    'few_shot_with_heur_hint_angle',
    'few_shot_with_heur_hint_distance',
    'few_shot_with_heur_hint_area',
    'few_shot_with_heur_hint_angle_distance',
    'few_shot_with_heur_hint_angle_area',
    'few_shot_with_heur_hint_distance_area',
    'few_shot_with_heur_hint_all',    
    'few_shot_with_heur_value_angle',
    'few_shot_with_heur_value_distance',
    'few_shot_with_heur_value_area',
    'few_shot_with_heur_value_angle_distance',
    'few_shot_with_heur_value_angle_area',
    'few_shot_with_heur_value_distance_area',
    'few_shot_with_heur_value_all',
]
## evaluate on a subset
np.random.seed(100)
index=np.random.randint(0, 3069, 1000)

### LLMs

In [ ]:
results=[]
models=['llama3', '4o_mini', 'qwen_plus', 'qwen_max','4o']
for model in models:
    print(f'Model: {model}...')
    for config in configs:
        pred=np.load(f'base/{model}/{model}_{config}.npy')
        pred=post_processing(pred)        
        if len(pred)==1000:
            metrics=metric_calculation(pred, gt)
        else:
            metrics=metric_calculation(pred[index], gt)
        results.append([config, model, metrics[0], metrics[1]])
results=pd.DataFrame(results, columns=['config', 'model', 'acc', 'f1'])

Model: llama3...


In [11]:
results[results['model']=='qwen_plus']

,config,model,acc,f1
58,zero_shot_no_heur,qwen_plus,0.603,0.392110
59,zero_shot_with_heur_hint_angle,qwen_plus,0.645,0.496562
60,zero_shot_with_heur_hint_distance,qwen_plus,0.609,0.405572
61,zero_shot_with_heur_hint_area,qwen_plus,0.603,0.385425
62,zero_shot_with_heur_hint_angle_distance,qwen_plus,0.676,0.564495
63,zero_shot_with_heur_hint_angle_area,qwen_plus,0.669,0.555690
64,zero_shot_with_heur_hint_distance_area,qwen_plus,0.633,0.477933
65,zero_shot_with_heur_hint_all,qwen_plus,0.705,0.631338
66,zero_shot_with_heur_value_angle,qwen_plus,0.946,0.942522
67,zero_shot_with_heur_value_distance,qwen_plus,0.732,0.656840
